In [1]:
import csv
import random
actorsById = {}
with open("name.short.tsv", 'r') as actors:
    reader = csv.DictReader(actors, delimiter='\t')
    for row in reader:
        actorsById[row['id']] = row['name']

filmsByTitle = {}
with open("combined.tsv", 'r') as basics:
    reader = csv.DictReader(basics, delimiter='\t')
    for row in reader:
        filmsByTitle[row['title']] = row
        filmsByTitle[row['title']]['genres'] = row['genres'].split(',')
        filmsByTitle[row['title']]['principals'] = row['principals'].split(',')

In [2]:
def get_random_film():
    r = random.randint(0, len(filmsByTitle))
    randomFilm = list(filmsByTitle.values())[r]
    return randomFilm

In [94]:
def get_list_percentage_similarity(listA, listB):
    return 100 * (len(set(listA)&set(listB)) / len(set(listA)|set(listB)))

In [110]:
def test_film(x):
    global current_results
    global random_film
    if x not in filmsByTitle:
        return
    if filmsByTitle[x] == random_film:
        return "Success!"

    guess_film = filmsByTitle[x]
    genre_diff = get_list_percentage_similarity(guess_film['genres'], random_film['genres'])
    principals_diff = get_list_percentage_similarity(guess_film['principals'], random_film['principals'])
    year_diff = int(guess_film['startYear']) - int(random_film['startYear'])
    guess_principal_names = ",".join([actorsById.get(p, "") for p in guess_film['principals']])
    guess_genres = ",".join(guess_film['genres'])
    result = f"{x} - \nprincipal matches={principals_diff}%\n{guess_principal_names}\ngenre matches={genre_diff}%\n{guess_genres}\nyear difference={year_diff}"
    current_results += result + "\n\n"
    print(current_results)

In [111]:
random_film = get_random_film()
print(f"cheat, answer is {random_film['title']}")
current_results = ""

cheat, answer is Sinais de Vida


In [112]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
films = ["-"]
films.extend(sorted(filmsByTitle.keys()))
interact(test_film, x=films)

interactive(children=(Dropdown(description='x', options=('-', '1984', '71 Fragments of a Chronology of Chance'…

<function __main__.test_film(x)>